<a href="https://colab.research.google.com/github/solanods/pandas/blob/master/alteracoes_rmops2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

from google.colab import drive
drive.mount('/content/drive')

from datetime import date
from re import search

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Carregamento dos dados
arquivo_antigo = '/content/PRMS_10_OUT_2022ontem.xlsm'
arquivo_recent = '/content/PRMS_10_OUT_2022.xlsm'

df_original = pd.read_excel(arquivo_antigo, sheet_name='RMOP')
df_alterado = pd.read_excel(arquivo_recent, sheet_name='RMOP')


colunas_alvo = list(range(327, 358))
df_escalas = pd.read_excel(arquivo_recent, sheet_name='DADOS', usecols=colunas_alvo)
df_escalas = df_escalas.loc[5:200]

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Funcao limpeza de dados

def tratarDataframe(df):
  
  # Selecionar colunas de interesse
  df = df.iloc[:,[3,5,6,7,8,11,16,17,18,19]]

  # Renomear colunas
  df = df.rename({'Unnamed: 3': 'DIA',
                  'Unnamed: 5': 'PREVISTO',
                  'Unnamed: 6': 'ESCALA',
                  'Unnamed: 7': 'TOTVS',
                  'Unnamed: 8': 'JORNADA',
                  'Unnamed: 11': 'ENTRADA',
                  'Unnamed: 16': 'CODIGO',
                  'Unnamed: 17': 'DESCRICAO',
                  'Unnamed: 18': 'OBSERVACOES',
                  'Unnamed: 19': 'RE_SGT'}, axis = 1)

  # Remover nan
  df = df.fillna('-')

  # Obter lista com nome de todos operadores
  operadores = []
  for linha in df.index:
    if search ('[A-Z]+', str(df['DIA'][linha])):
      operadores.append(df['DIA'][linha])

  operadores = [operador for operador in operadores if operador.isupper()]

  # Extrair rmop
  i = 3
  j =38
  R = []
  df['NOME'] = ''
  df['RE'] = ''
  for operador in operadores:
    df['NOME'].loc[i:j] = operador
    rmop_operador = df.loc[i:j]
    R.append(rmop_operador)
    df['RE'].loc[i:j] = df['ENTRADA'].loc[i:j].iloc[0]
    i+=66
    j+=66

  return df

In [ ]:
df_original = tratarDataframe(df_original)
df_alterado = tratarDataframe(df_alterado)

In [ ]:
# Criar novo df somente com os índices modificados, ou seja, as diferenças entre ambos
rmops_alterado = pd.concat([df_original,df_alterado]).drop_duplicates(keep=False)

In [ ]:
# Resultado mostra os indices originais e alterados em sequencia
rmops_alterado

,DIA,PREVISTO,ESCALA,TOTVS,JORNADA,ENTRADA,CODIGO,DESCRICAO,OBSERVACOES,RE_SGT,NOME,RE
566,31,TRAB,R1,15-O,02T,14:00:00,-,-,-,-,ANA CAROLINA,2922
962,31,FOL,A4,11-K,FOL,,D,Disponível para HE,-,252,ARAUJO,3203
1092,29,FOL,JA,9-I,TMR,13:15:00,C129,Troca Jorn,por 15,2923,ARTUR,2934
1093,30,FOL,JA,9-I,TMR,13:15:00,C129,Troca Jorn,p/dia 25 nec oper,2934,ARTUR,2934
1156,27,FOL,FA,1-A,MNH,05:30:00,C129,Troca Jorn,por 2,2923,AZAMBUJA,587
1621,30,TRAB,C4,7-G,15M,05:10:00,-,-,-,-,CARBONERA,1486
2874,29,FOL,I4,13-M,FOL,,D,Disponível para HE,-,-,DIAS,3210
2875,30,FOL,I4,13-M,FOL,,D,Disponível para HE,-,-,DIAS,3210
2876,31,FOL,I4,13-M,FOL,,D,Disponível para HE,-,-,DIAS,3210
3203,28,FOL,J2,13-M,FOL,,D,Disponível para HE,-,-,EDNA,3094


In [ ]:
# Criar campo data
data_atual = date.today()
rmops_alterado['DATA_COLETA_DADO'] = data_atual

In [ ]:
# Salvar no google drive
rmops_alterado.to_excel(f'/content/drive/MyDrive/alteracoesrmop{data_atual}.xlsx')

In [ ]:
# Numero de alteracoes
num_alteracoes = len(rmops_alterado.index)/2

In [ ]:
# String para arquivar
dados = str(data_atual) + '   N°_ALTERACOES: ' + str(int(num_alteracoes))

In [ ]:
dados

'2022-10-28   N°_ALTERACOES: 35'

In [ ]:
# Salvar em arquivo texto
file = open('/content/drive/MyDrive/alteracoes_rmop_contador.txt', 'a')
file.write(dados + '\n')
file.close()

In [ ]:
# Verificar entre jornadas
  
for i in df_alterado.index:
    if search ('\d{2}T', str(df_alterado['JORNADA'][i])):
      if search ('\d{2}M', str(df_alterado['JORNADA'][i+1])):
        
        print('VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS\n')
        
        print('Dia', 
              df_alterado['DIA'][i], 
              df_alterado['NOME'][i], 
              df_alterado['JORNADA'][i],
              df_alterado['ENTRADA'][i])
        
        print('Dia', 
              df_alterado['DIA'][i+1], 
              df_alterado['NOME'][i+1], 
              df_alterado['JORNADA'][i+1],
              df_alterado['ENTRADA'][i+1])

VERIFICAR INTERVALO DE 11 HORAS ENTRE JORNADAS

Dia 24 ANA CAROLINA 01T 12:00:00
Dia 25 ANA CAROLINA 05M 05:30:00


In [ ]:
# Busca escalas dos semanais

print('ESCALAS DOS SEMANAIS SAINDO OU RETORNANDO DE FÉRIAS \n')
for i in range(len(df_escalas.index)):
  if len(df_escalas.iloc[i].unique()) > 2:
    if search('[X|Y]\d{2}', str(df_escalas.iloc[i])):
      print(df_escalas.iloc[i].unique())

ESCALAS DOS SEMANAIS SAINDO OU RETORNANDO DE FÉRIAS 

['ANDRADE' 'X11' 'C3']
['ARRUE' 'N5' 'FER' 'FOL' 'ASS' 'Y12']
['CESAR' 'FER' 'ASS' 'FOL' 'Y08' 'L4']
['CLAUDETE' 'FER' 'PCA' 'FOL' 'X15']
['CORREA' 'X06' 'A7' 'E5']
['DIAS' 'X08' 'G6' 'B6' 'D7' 'I4']
['FREITAS' 'FER' 'Y09']
['FURLAN' 'I5' 'FER' 'PCA' 'FOL' 'X17']
['INES MIRANDA' 'FER' 'ASS' 'X12' 'H2']
['ISABEL' 'X03' 'FER']
['LAURI' 'Y04' 'N5' 'FOL']
['LEOCY' 'X10' 'G2']
['LETICIA JAQUES' 'X07' 'E7' 'I5']
['PRANDINI' 'X09' 'FER' 'PCA' 'X16']
['ROGER' 'X05' 'E6' 'FOL']
['SAUL' 'FER' 'ASS' 'Y11']
['SIMONE' 'FER' 'Y10']
['SOARES' 'FER' 'ASS' 'FOL' 'X13' 'E3']
['TAVARES' 'X04' 'A6' 'A2' 'FOL']
['TUBINO' 'Y07' 'P1']
['VERGARA' 'X02' 'FOL']
['VIEIRA' 'Y06' 'L6' 'Q3']
['VINICIUS' 'Y05' 'R7' 'M7' 'M1']
['VIVIANE' 'FER' 'FOL' 'PCA' 'X14']
